In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pickle

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer

from joblib import load

import glob
import os

import keras

from numpy import genfromtxt

2023-01-02 16:14:25.051170: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-02 16:14:26.115235: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-02 16:14:26.115455: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-02 16:14:26.115461: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:

fb = pd.read_json ("fb.json")
fb

,postid,post,header,date,user,userid,url,quote,quoteid,quotetext,shorturl,media_type,num_am,num_mm,num_all,calc_quoteid
0,18655433,Sänds i SVT 1 ikväll 090910 klockan 19.30 http...,Skolfront i SVT om kulturkrockar i skolan/elev...,2009-09-10 13:11:00,knullkungen,74027,[http://www.ur.se/Skolfront/Titta-pa-Skolfront...,[],[],[],[ur.se],[misc],0,0,1,[]
1,18655506,Det är väl bra att problemet kommer fram i dag...,Skolfront i SVT om kulturkrockar i skolan/elev...,2009-09-10 13:15:00,kaiser chief,179428,[],[],[],[],[None],[NA],0,0,1,[]
2,18655849,Min kommentar är att integrationsministern int...,Skolfront i SVT om kulturkrockar i skolan/elev...,2009-09-10 13:41:00,8d8,232950,[],[],[],[],[None],[NA],0,0,1,[]
3,18655896,Yepp webtv kommer i efterhand Sabuni är f.ö. i...,Skolfront i SVT om kulturkrockar i skolan/elev...,2009-09-10 13:44:00,knullkungen,74027,[],[8d8],[],[Ursprungligen postat av 8D8 Min kommentar är ...,[None],[NA],0,0,1,[18655849]
4,18658705,Maja Lundgrens uttalande visar hur långt det h...,Skolfront i SVT om kulturkrockar i skolan/elev...,2009-09-10 16:55:00,rammsberg,132764,[],[knullkungen],[],[Ursprungligen postat av Knullkungen Sänds i S...,[None],[NA],0,0,1,[18655433]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002631,18677852,Kan du inte försöka bidra till trådarna iställ...,Hur ska vi stoppa dessa finnar som väller in o...,2009-09-11 18:03:00,alcoholocaust-,346534,[],[lobuno],[],[Ursprungligen postat av Lobuno Men är det int...,[None],[NA],0,0,1,[18677691]
2002632,18678232,Korrekt vi vill inte ha hit finska pedofiler o...,Hur ska vi stoppa dessa finnar som väller in o...,2009-09-11 18:26:00,kroppskultur,240960,[],[straybullet],[],[Ursprungligen postat av Straybullet Varför pr...,[None],[NA],0,0,1,[18665476]
2002633,18678440,Appropå TS så vart tog han vägen? Har inte set...,Hur ska vi stoppa dessa finnar som väller in o...,2009-09-11 18:39:00,flashen1961,310716,[],[kroppskultur],[],[Ursprungligen postat av kroppskultur Korrekt ...,[None],[NA],0,0,1,[18678232]
2002634,18680988,"Ok, du har en blattefierad whigger-finne som f...",Hur ska vi stoppa dessa finnar som väller in o...,2009-09-11 21:03:00,nalin_pekgul,236141,[http://www.aftonbladet.se/senastenytt/ttnyhet...,[b0gfetto],[],[Ursprungligen postat av b0gfetto Dessa finnar...,"[aftonbladet.se, frianyheter.wordpress.com]","[mm, na]",0,1,2,[18645292]


In [3]:
#clean strings

text = fb['post']

#remove html markup
text = text.str.replace('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});',' ')

#remove tabs
text= text.str.replace('[ |\\t]{2,}', ' ', case=False, regex = True)

#remove urls 
text= text.str.replace('http\\S+|www\\.\\S+', ' ', case=False, regex = True)

#remove urls 
text= text.str.replace('\\S+\\.se|\\S+\\.com|\\S+\\.org', ' ', case=False, regex = True)

#remove e-mails
text= text.str.replace('\\S+@\\S+', ' ', case=False, regex = True)

#remove additional links
text= text.str.replace('bit\\.ly\\S+', ' ', case=False, regex = True)
text= text.str.replace('goo\\.gl\\S+', ' ', case=False, regex = True)
text= text.str.replace('youtu\\.be\\S+', ' ', case=False, regex = True)

#remove characters after semicolon
text= text.str.replace('\\:\\S+', '', case=False, regex = True)

#replace ... with .
text = text.str.replace('\\.\\.+','\\.', regex = True)

text = text.str.replace(' +',' ').str.strip()

text = text.tolist()

/tmp/ipykernel_3916/454220391.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});',' ')
/tmp/ipykernel_3916/454220391.py:31: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace(' +',' ').str.strip()


In [4]:
model = SentenceTransformer('KBLab/sentence-bert-swedish-cased')

In [5]:
column_ids = []
for i in range(0, 768):
    column_ids.append(str(i))

In [6]:
#generate features for the unseen data using BERT
import csv

embeddings = []
for i in range(0, len (text)):
    
    
    encoding = model.encode(text[i])
    embeddings.append(encoding)
    
    if (i>0) & (i % 200000 == 0):
        print(i)
        
        df = pd.DataFrame(embeddings, columns = column_ids)
        df.to_csv("embeddings" + str(i) + ".csv")

        embeddings = []
    
    elif (i == (len(text)-1)):
        
        df = pd.DataFrame(embeddings, columns = column_ids)
        df.to_csv("embeddings" + str(i) + ".csv")

200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
2000000


In [2]:
#load previously fine-tuned model to classify on- and off-topic messages
tf_model = keras.models.load_model('model_bert_topic_offtopic')

2023-01-02 16:14:37.122380: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_SYSTEM_DRIVER_MISMATCH: system has unsupported display driver / cuda driver combination
2023-01-02 16:14:37.122410: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: NO-1DAE17E7A
2023-01-02 16:14:37.122415: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: NO-1DAE17E7A
2023-01-02 16:14:37.122548: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.108.3
2023-01-02 16:14:37.122566: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  525.60.11  Release Build  (dvs-builder@U16-T01-9-2)  Wed Nov 23 23:04:11 UTC 2022
GCC ve

In [17]:
#classify messages

all_files = ['embeddings200000.csv', 'embeddings400000.csv', 'embeddings600000.csv', 'embeddings800000.csv', 
             'embeddings1000000.csv', 'embeddings1200000.csv', 'embeddings1400000.csv', 'embeddings1600000.csv',  
            'embeddings1800000.csv', 'embeddings2000000.csv', 'embeddings2002635.csv']

class_names = ['off-topic', 'topic']
labels = []
values = []

for filename in all_files:
    
    print(filename)
    embeddings = pd.read_csv(filename) 
    embeddings = embeddings.drop (columns = "Unnamed: 0").to_numpy()
    embeddings_X = np.expand_dims(embeddings, axis=1)
    predictions = tf_model(embeddings_X, training=False)
    print(len(predictions))
   
    for i, logits in enumerate(predictions):
       
        class_idx=np.argmax(logits.numpy())
        name = class_names[class_idx]
        val = np.max(logits.numpy())
    
        labels.append(name)
        values.append (val)


embeddings200000.csv
200001
embeddings400000.csv
200000
embeddings600000.csv
200000
embeddings800000.csv
200000
embeddings1000000.csv
200000
embeddings1200000.csv
200000
embeddings1400000.csv
200000
embeddings1600000.csv
200000
embeddings1800000.csv
200000
embeddings2000000.csv
200000
embeddings2002635.csv
2635


In [21]:
#CLASSIFY STANCEs WITH THE FINE-TUNED BERT MODEL¶

#load model to classify stances (negative and non-negative)
clf = load("trained_classifier_model_mlp-sbert-features_negative_non-negative.joblib")

/home/veronica/.local/lib/python3.10/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator LabelBinarizer from version 1.1.1 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/veronica/.local/lib/python3.10/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator MLPClassifier from version 1.1.1 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [22]:
#classify messages
all_files = ['embeddings200000.csv', 'embeddings400000.csv', 'embeddings600000.csv', 'embeddings800000.csv', 
             'embeddings1000000.csv', 'embeddings1200000.csv', 'embeddings1400000.csv', 'embeddings1600000.csv',  
            'embeddings1800000.csv', 'embeddings2000000.csv', 'embeddings2002635.csv']

stance_names = ['neg', 'nonneg']
stance_probs = []
stance_labels = []

for filename in all_files:
    
    print(filename)
    embeddings = pd.read_csv(filename) 
    embeddings = embeddings.drop (columns = "Unnamed: 0").to_numpy()
    
    labels_pred = clf.predict(embeddings)
    probs_pred = clf.predict_proba(embeddings) 
   
    for i, logits in enumerate(probs_pred):
     
        class_idx=np.argmax(logits)
        name = stance_names[class_idx]    
        val = np.max(logits)
    
        stance_labels.append(name)
        stance_probs.append (val)


embeddings200000.csv
embeddings400000.csv
embeddings600000.csv
embeddings800000.csv
embeddings1000000.csv
embeddings1200000.csv
embeddings1400000.csv
embeddings1600000.csv
embeddings1800000.csv
embeddings2000000.csv
embeddings2002635.csv


In [25]:
df = pd.DataFrame(
    {'tot_label': labels,
     'tot_prob': values,
     'stance_label': stance_labels,
     'stance_prob' : stance_probs
    })

df.to_csv("flashback_stance_classification.csv")

In [ ]:
#add data on user stances to the df

fb = pd.merge(fb, df, left_index = True, right_index=True)

fb['stance_label'] = np.where(fb['tot_label'] == "off-topic", fb['tot_label'], fb['stance_label'])
fb['stance_prob'] = np.where(fb['stance_label'] == "off-topic", fb['tot_prob'], fb['stance_prob'])
fb.to_json("fb.json")